#testing out the NY Times API

In [1]:
from urllib2 import Request, urlopen, URLError
import json
import psycopg2

In [2]:
#start with a function that makes the URI call for a given isbn number
def make_URI(isbn_number):
    key = 'ec4e2a5e373e93f850f9be6c420bb73a:2:73271883'
    prefix = 'http://api.nytimes.com/svc/books/v3/lists/'
    best_sellers = 'best-sellers/history.json?author='
    api_key_string = "&api-key="  + key
    URI = prefix + best_sellers + str(isbn_number) + api_key_string
#    print URI
    return URI

In [3]:
#function that pulls the json file from the nytimes api
def pull_nytimes_data(URI):
    request = Request(URI)
    response = urlopen(request)
    parsed_response = json.loads(response.read())
#    print parsed_response
    return parsed_response

In [4]:
def is_it_on_the_list(parsed_json):
    membership = parsed_json['num_results']
    return membership

In [5]:
##this takes the author and title since they're in a different part of the json tile
def extract_author_and_title(input):
    author = input['results'][0]['author']
    title = input['results'][0]['title']
    print author
    print title
    return author,title

In [6]:
#this function goes through each of the entries in the bestselling list and makes it alist ad put it into a list of lists
def make_lists_of_data(input,isbn):
    entry_list = [None] * len(input['results'][0]['ranks_history'])

    #start the entry list with the outer values, title and author
    title = input['results'][0]['title'].encode('ascii', 'ignore')
    isbn_10 = str(isbn) 
    author = input['results'][0]['author'].encode('ascii', 'ignore')    
    description = input['results'][0]['description']
    
    
    counter = 0
    for item in input['results'][0]['ranks_history']:
        entry = [title, isbn, author]
        
        rank = item['rank']
        date = str(item['bestsellers_date'])
        weeks_on_list = item['weeks_on_list']
        book_list = str(item['list_name'])
        
        entry = entry + [book_list, rank, date, weeks_on_list] + [str(description)]

        contributor = input['results'][0]['contributor'].encode('ascii', 'ignore')
        publisher = input['results'][0]['publisher'].encode('ascii', 'ignore')
        
        entry = entry + [contributor, publisher]
        entry_list[counter] = entry
        counter = counter + 1
    return entry_list

In [7]:
#this executemanies the list of lists into the database 
def put_in_database(results_as_a_list):    
    con = None
    con = psycopg2.connect(database='nytimes') 
    cur = con.cursor()
    query = "INSERT INTO books (title, isbn_10, author, list, rank, date, weeks_on_list, description, \
        contributor, publisher) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.executemany(query, results_as_a_list)
    con.commit()
    con.close()
    return

In [8]:
#start off with the isbn of the book you want to put into the database
#isbn = '0553418025' #THE MARTIAN
#isbn = '1476783454' #the survivor
#remember to enter the isbn as a string
def put_book_into_database(isbn):    
    current_URI = make_URI(isbn)
    json_data = pull_nytimes_data(current_URI)
    if is_it_on_the_list(json_data) == 0:
        print "not on the list"
        return 
    ranking_results = make_lists_of_data(json_data,isbn)
    put_in_database(ranking_results)
    return



In [10]:
#put_book_into_database(1476746583)
put_book_into_database('Grisham')


#isbn_list = ['1476727392','0062225243','0312510780','0385539436','1419717014','1419717014','1607747308']
#for book in isbn_list:
#    print(str(book))
#    put_book_into_database(book)

In [177]:
#fetch isbn numbers from the database, then get their histories
con = psycopg2.connect("dbname='nytimes'") 
cur = con.cursor()    
cur.execute("SELECT isbn_10 FROM books")
rows = cur.fetchall()
con.close()

In [178]:
isbn_list = list()
for row in rows:
    if row[0] != 'None':
        if row[0] != '':    
            isbn_list.append(row[0])
isbn_list = list(set(isbn_list))        

In [179]:
for book in isbn_list:
    print(str(book))
    put_book_into_database(book)

1476757801


In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
